In [2]:
# import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
from matplotlib.pyplot import figure
from skimage.transform import iradon
import matplotlib
import cProfile, pstats, io
from pstats import SortKey
%matplotlib inline



#Chelsea's stripping method
def generateResponseFunctionSingleton(dir_response_fn, filename_1, filename_2, min_energy, max_energy,
                                                        energy_bin_width):
    '''
    helper function for loading all response functions right away, so only have to do this once especially if looping
    over XFCT data
    :param dir_response_fn: str
    :param filename_1: str, "responsefunction"
    :param filename_2: str, "keV5x5.npy"
    :param min_energy: float
    :param max_energy: float or int
    :param energy_bin_width: float
    :return: giant object array
    '''
    MASTER_RESPONSE = np.zeros(int(1000), dtype=object)
    for j in np.arange(min_energy, max_energy+0.2, step=energy_bin_width):
        energy, edep = np.load(dir_response_fn + filename_1 + str(round(j,1)) + filename_2)       
        MASTER_RESPONSE[int(j * 5)] = edep/4.5
    return MASTER_RESPONSE

def stripEnergySpectrum(spectrum, MASTER_RESPONSE, min_energy, max_energy, width_energy, dirfile_feae,
                        number_of_primaries):
    '''
    Corrects a energy-deposited spectrum using the stripping method and returns the spectrum of x-rays that were
    incident on the CdTe crystal
    :return: corrected x-ray spectrum of len(spectrum)
    '''

    # load the full energy absorption efficiency
    topas_energy, topas_abseff = np.load(dirfile_feae)
    topas_abseff=topas_abseff*100

    max_index = max_energy/width_energy
    n_true = np.zeros(int(max_index))

    for i in np.arange(min_energy, max_energy, step=width_energy)[::-1]:
        i_index = int(i/width_energy)
        n_det = np.take(spectrum, i_index)
        j = i + width_energy
        j_index = int(j/width_energy)
        
        while j_index < max_index:
            edep = np.take(MASTER_RESPONSE, j_index)
            energy = np.arange(0, j, step=width_energy)

            edep_norm = edep / float(number_of_primaries)
            energy = np.around(energy, decimals=1)
            i = np.around(i, decimals=1)
            resp = np.take(edep_norm, np.where(np.equal(energy, i))[0][0])

            n_det = n_det - (resp * np.take(n_true, j_index))
            j_index = j_index + 1
            j = j + width_energy

        topas_energy = np.around(topas_energy, 1)
        i_width = np.around(i + width_energy, 1)
        index_feae = np.where(np.equal(topas_energy, i_width))[0][0]
        if n_det < 0:
            n_true[i_index] = 0
        else:
            n_true[i_index] = 100 * n_det / np.take(topas_abseff,  index_feae)

    return n_true


data_directory = "Desktop/topas_sim_files/"

# next, specify the directory of the response functions, their file name convention, and the energy range.
dir_response_fn = data_directory + "response_functions/"
filename_1 = "responsefunction_" #filename in two parts because that's where energy inserted
filename_2 = "keV3x3.npy" #if you name it better, this could just be ".npy"
min_energy = 10.0 # keV
max_energy = 120  # keV
energy_bin_width = 0.2 #keV, this is what I went with
num_primaries = 5e5 # these were the number of primaries used to generate the response functions
feae_file = data_directory + "abseff3x3.npy"

MASTER_RESPONSE = generateResponseFunctionSingleton(dir_response_fn, filename_1, filename_2, min_energy, max_energy,
                                                        energy_bin_width)

old_x=np.linspace(0,150.8073,1024)
new_x=np.arange(0,120,0.2)


#Air scan
# counts = np.genfromtxt("Desktop/CT imaging/30-6-2021/airscan.mca", skip_header=16,skip_footer=71)
# new_counts=np.interp(new_x,old_x,counts)
# airscan_counts1=stripEnergySpectrum(new_counts, MASTER_RESPONSE, min_energy, max_energy, energy_bin_width,
#                                             feae_file, num_primaries)
# counts = np.genfromtxt("Desktop/CT imaging/30-6-2021/angle_128_acq_01.mca", skip_header=16,skip_footer=71)
# new_counts=np.interp(new_x,old_x,counts)
# airscan_counts2=stripEnergySpectrum(new_counts, MASTER_RESPONSE, min_energy, max_energy, energy_bin_width,
#                                             feae_file, num_primaries)

# tot_counts=[]
# master_counts=[]
# spectrum=[]
# for i in np.arange(0,360,8):
#     for j in np.arange(1,97):
#         counts = np.genfromtxt("Desktop/CT imaging/30-6-2021/angle_{:d}_acq_{:02d}.mca".format(i,j), skip_header=16,skip_footer=71)
#         new_counts=np.interp(new_x,old_x,counts)
#         stripped_spectrum = stripEnergySpectrum(new_counts, MASTER_RESPONSE, min_energy, max_energy, energy_bin_width,feae_file, num_primaries)
#         if i<=120:
#             I_I0=np.divide(stripped_spectrum,airscan_counts1, out=stripped_spectrum, where=airscan_counts1!=0)
#         else:
#             I_I0=np.divide(stripped_spectrum,airscan_counts2, out=stripped_spectrum, where=airscan_counts2!=0)
#         spectrum.append(-np.log(I_I0,out=np.zeros_like(I_I0), where=(I_I0!=0)))
#         tot_counts.append(spectrum)
#         spectrum=[]
#     master_counts.append(tot_counts)
#     tot_counts=[]

# master_counts=np.array(master_counts)
# master_counts=np.squeeze(master_counts)

master_counts[4].pop()
master_counts[4].insert(0,0)
master_counts[38].pop()
master_counts[38].insert(0,0)
# np.save("Desktop/CT imaging/master_counts_strip_8deg.npy",master_counts)

figure(1,figsize=(9,9))
plt.imshow(master_counts[3])
# master_counts=np.load("Desktop/CT imaging/master_counts_stripped.npy")

# bin_width=1
# bins=np.arange(20,110,bin_width)
# master_binned=np.zeros((36,90,90))
# for k in np.arange(0,len(bins)):
#     for i in np.arange(0,36):
#         for j in np.arange(0,90):
#             master_binned[i][j][k]=master_counts[i][j][int(np.around((k+20)/0.2)):int(np.around(((k+21)/0.2)))].mean()
# master_1kev=master_binned
# #np.save("Desktop/CT imaging/master_strp_3kev.npy",master_3kev)

# figure(1,figsize=(9,8))
# plt.plot(bins,master_1kev[7][8])


AttributeError: 'numpy.ndarray' object has no attribute 'pop'